<a href="https://colab.research.google.com/github/Jepees/StepAhead-Nuevo/blob/main/scripts/2_data_merger_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta
import re

In [ ]:
brands = ['910', 'Ortuseight', 'Mills', 'Nuevo']

datas = {
            'df_910'        : pd.read_csv("https://raw.githubusercontent.com/Jepees/StepAhead-Nuevo/refs/heads/main/data/Tokopedia_review_910.csv"),
            'df_ortuseight' : pd.read_csv("https://raw.githubusercontent.com/Jepees/StepAhead-Nuevo/refs/heads/main/data/Tokopedia_review_Ortuseight.csv"),
            'df_mills'      : pd.read_csv("https://raw.githubusercontent.com/Jepees/StepAhead-Nuevo/refs/heads/main/data/Tokopedia_review_Mills.csv"),
            'df_nuevo'      : pd.read_csv("https://raw.githubusercontent.com/Jepees/StepAhead-Nuevo/refs/heads/main/data/Tokopedia_review_Nuevo.csv"),
        }
data_map = pd.read_csv("https://raw.githubusercontent.com/Jepees/StepAhead-Nuevo/refs/heads/main/data/database_sepatu.csv")

for brand in brands:
    datas[f'df_{brand.lower()}']['Brand'] = brand

In [ ]:
df = pd.concat(list(datas.values()), ignore_index=True)
df.sample(5)

,nama barang,kategori,harga,date,user,varian,rating,review,bantu,Brand
4184,Ortuseight Sepatu Futsal Jogosala Strive White...,futsal,Rp399.000,4 hari lalu,P***p,White Lime - 41,bintang 5,Sesuai pesanan barang original punya mantapp ...,0 orang terbantu,Ortuseight
7417,Ortuseight Sepatu Sneakers Berlin Beige - Beig...,sneakers,Rp599.000,2 bulan lalu,M***f,Beige - 39,bintang 5,Good,0 orang terbantu,Ortuseight
1003,910 nineten IREZUMI HOVER sepatu lari - BIRU-T...,Lari,Rp664.905,Lebih dari 1 tahun lalu,I***y,43.0,bintang 4,Good,0 orang terbantu,910
13206,NUEVO - Wilmax Putih Purple Sepatu Lari Olahra...,Lari,Rp157.500,5 hari lalu,R***n,Putih/Ungu/Oren - 39,bintang 5,"Masyaallah aku nggk bisa berkata "" sepatunya k...",0 orang terbantu,Nuevo
5397,Ortuseight Sepatu Running Hyperblast Encore Bl...,Lari,Rp549.000,1 bulan lalu,S***v,Black White - 43,bintang 5,Keren.. 👍👍,0 orang terbantu,Ortuseight


In [ ]:
# Merge berdasarkan 'nama barang' agar dapat 'nama fix' dan kategori bersih
df = df.merge(
    data_map[['nama barang', 'nama fix', 'kategori']],
    on='nama barang',
    how='left',
    suffixes=('', '_baru')
)

df['nama barang'] = df['nama fix']
df['kategori'] = df['kategori_baru']


df = df.drop(columns=['nama fix', 'kategori_baru'])

df

,nama barang,kategori,harga,date,user,varian,rating,review,bantu,Brand
0,nineten haze strike prime,sepatu lari,Rp550.905,2 hari lalu,A***s,42.0,bintang 5,Sepatunya ok dan pas dengan ukuran.. Nyampenya...,0 orang terbantu,910
1,nineten haze strike prime,sepatu lari,Rp550.905,1 hari lalu,Z***i,42.0,bintang 5,barang original enak di pakek ngebut sepatunya,0 orang terbantu,910
2,nineten haze strike prime,sepatu lari,Rp550.905,1 minggu lalu,R***i,45.0,bintang 5,Bagus nyaman melebihi ekspektasi saya,0 orang terbantu,910
3,nineten haze strike prime,sepatu lari,Rp550.905,2 minggu lalu,V***u,41.0,bintang 5,Produknya mantapppplll,1 orang terbantu,910
4,nineten haze strike prime,sepatu lari,Rp550.905,3 hari lalu,Y***e,40.0,bintang 5,Pengiriman cepat,0 orang terbantu,910
...,...,...,...,...,...,...,...,...,...,...
13917,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,Lebih dari 1 tahun lalu,A***e,38,bintang 4,NaN,0 orang terbantu,Nuevo
13918,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,Lebih dari 1 tahun lalu,D***i,40,bintang 5,NaN,0 orang terbantu,Nuevo
13919,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,Lebih dari 1 tahun lalu,d***a,42,bintang 5,Lumyan,0 orang terbantu,Nuevo
13920,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,Lebih dari 1 tahun lalu,r***n,41,bintang 3,NaN,0 orang terbantu,Nuevo


In [ ]:
# menghapus baris duplikat

print(f'data duplikat sebelum penghapusan: {df.duplicated().sum()}')
df.drop_duplicates(inplace=True)
print(f'data duplikat setelah penghapusan: {df.duplicated().sum()}')

data duplikat sebelum penghapusan: 9
data duplikat setelah penghapusan: 0


In [ ]:
# menghapus baris yang kolom 'review' nya kosong

print(f'total data sebelum penghapusan review kosong: {df.shape[0]}')
df = df[df['review'].notna()].reset_index(drop=True)
print(f'total data setelah penghapusan review kosong: {df.shape[0]}')

total data sebelum penghapusan review kosong: 13913
total data setelah penghapusan review kosong: 13455


In [ ]:
df.sample(5)

,nama barang,kategori,harga,date,user,varian,rating,review,bantu,Brand
5981,ortuseight catalyst liberte v4 fg,sepatu bola,Rp649.000,1 bulan lalu,O***y,Cyan Pink - 41,bintang 5,barangnya baguss kurir nya ramahh bagus banget...,1 orang terbantu,Ortuseight
12041,nuevo velocity,sepatu lari,Rp156.350,2 bulan lalu,M***a,Putih Biru - 41,bintang 5,barang nya bagus pas ukuran nya enak banget pa...,0 orang terbantu,Nuevo
11870,nuevo soars,sepatu lari,Rp181.500,3 bulan lalu,I***k,STABILO KUNING - 43,bintang 5,Sangat bgus,0 orang terbantu,Nuevo
5102,ortuseight hyperblast 2.2,sepatu lari,Rp749.000,1 bulan lalu,U***j,White Camo - 46,bintang 5,cocok untuk pemula,0 orang terbantu,Ortuseight
3703,ortuseight jogosala ascend,sepatu futsal,Rp499.000,5 bulan lalu,S***f,Black Camo - 43,bintang 5,Nice,0 orang terbantu,Ortuseight


In [ ]:
def ke_bulan_tahun(text):
    today = datetime.today()

    if not isinstance(text, str):
        return None
    text = text.lower().strip()
    tanggal = today

    if "hari ini" in text or "hari lalu" in text or "minggu lalu" in text:
        tanggal = today
    elif "bulan lalu" in text:
        jumlah = int(re.search(r"\d+", text).group())
        tanggal = today - timedelta(days=30 * jumlah)  # asumsi 30 hari per bulan
    elif "lebih dari 1 tahun" in text:
        tanggal = today - timedelta(days=450)
    elif "tahun lalu" in text:
        tanggal = today - timedelta(days=365)
    else:
        return None
    return tanggal.strftime('%B %Y')

def pisah_varian(text):
    if isinstance(text, str):
        if '-' in text:
            parts = text.split('-', maxsplit=1)
            varian = parts[0].strip()
            ukuran = parts[1].strip()
        else:
            if text.strip().isdigit():
                varian = None
                ukuran = text.strip()
            else:
                varian = text.strip()
                ukuran = None
    else:
        varian = None
        ukuran = None
    return pd.Series([varian, ukuran])

def bersihkan_ukuran(val):
    if not isinstance(val, str):
        val = str(val)

    # Ambil angka di kiri jika ada tanda '/'
    if '/' in val:
        val = val.split('/')[0].strip()

    # Buang titik/koma desimal
    val = re.sub(r'[,.].*', '', val)

    # Cek apakah sisa val adalah digit
    return int(val) if val.isdigit() else None


In [ ]:
df['date'] = df['date'].apply(ke_bulan_tahun)
df[['varian', 'ukuran']] = df['varian'].apply(pisah_varian)
# df['ukuran'] = pd.to_numeric(df['ukuran'], errors='coerce').astype('Int64')


df['rating'] = (
    df['rating']
    .str.replace('bintang ', '', regex=False)
    .astype('Int64')
)

df['harga_int'] = (
    df['harga']
    .str.replace('Rp', '', regex=False)      # hilangkan "Rp"
    .str.replace('.', '', regex=False)       # hilangkan titik pemisah ribuan
    .str.strip()                             # hilangkan spasi ekstra
    .astype(int)                             # ubah ke integer
)

df['terbantu'] = (
    df['bantu']
    .str.replace(' orang terbantu', '', regex=False)
    .astype('Int64')
)

df['ukuran'] = df['ukuran'].apply(bersihkan_ukuran)
df.sample(5)

,nama barang,kategori,harga,date,user,varian,rating,review,bantu,Brand,ukuran,harga_int,terbantu
4437,ortuseight mirage in,sepatu futsal,Rp299.000,April 2024,B***i,None,5,ok,0 orang terbantu,Ortuseight,43.0,299000,0
975,nineten irezumi hover,sepatu lari,Rp664.905,April 2024,K****,None,5,Aman...,0 orang terbantu,910,NaN,664905,0
9821,mills treximo omega,sepatu lari,Rp349.000,February 2025,e***5,Black/Black,5,Pas,0 orang terbantu,Mills,43.0,349000,0
4023,ortuseight jogosala strive,sepatu futsal,Rp399.000,May 2025,D***r,Beige Navy,5,Good buat sepatunya,0 orang terbantu,Ortuseight,42.0,399000,0
12031,nuevo velocity,sepatu lari,Rp156.350,February 2025,j***y,Hitam Gold,4,untuk cat nya kurang rapi tapi saya maklumi ka...,0 orang terbantu,Nuevo,39.0,156350,0


In [ ]:
df = df[[   'Brand', 'nama barang', 'kategori', 'harga', 'harga_int',
            'user', 'varian', 'ukuran',
            'date', 'review', 'rating', 'terbantu']]

df.sample(5)

,Brand,nama barang,kategori,harga,harga_int,user,varian,ukuran,date,review,rating,terbantu
2148,910,nineten sera,sepatu lari,Rp446.405,446405,M***d,None,NaN,April 2024,🔥,5,0
3791,Ortuseight,ortuseight jogosala courage,sepatu futsal,Rp499.000,499000,T***S,Black Gold,40.0,September 2024,"Kualitas bagus, harga worth it bgt, keren di p...",5,0
1958,910,nineten hiroshi rn - lite,sepatu lari,Rp645.905,645905,u***1,None,NaN,May 2025,"kaos kaki nya gaada, kabur sendiri kali ya apa...",5,0
5288,Ortuseight,ortuseight hyperblast encore,sepatu lari,Rp599.000,599000,M***r,Cyan Camo,44.0,July 2025,Bagus banget,5,0
2077,910,nineten kishi run 2.0,sepatu lari,Rp645.905,645905,a***h,None,NaN,December 2024,"Barang sesuai, bagus, keren, semoga awet",5,0


1. tren pembelian 4 brand, fokus ke sepatu nuevo
2. sentimen produk2 nuevo
3. negatif ke arah apanya?
4. topic di sentimen positif di brand lain
5. sepatu 5-10 sepatu paling populer di brand lain (tanpa nuevo)
6. kenapa itu populer?
7. rata2 harga per kategori sepatu antara masing2 brand

In [ ]:
df.loc[df['nama barang'].isna(), 'nama barang'] = 'nineten easy train'
df.loc[df['kategori'].isna(), 'kategori'] = 'sepatu lari'

In [ ]:
for col in df.columns:
    print(f'kolom {col}: {df[col].isna().sum()}')

kolom Brand: 0
kolom nama barang: 0
kolom kategori: 0
kolom harga: 0
kolom harga_int: 0
kolom user: 1
kolom varian: 3959
kolom ukuran: 3204
kolom date: 0
kolom review: 0
kolom rating: 0
kolom terbantu: 0


In [ ]:
df.to_csv('data_clean_merge.csv', index=False)
df

,Brand,nama barang,kategori,harga,harga_int,user,varian,ukuran,date,review,rating,terbantu
0,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,A***s,None,NaN,July 2025,Sepatunya ok dan pas dengan ukuran.. Nyampenya...,5,0
1,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,Z***i,None,NaN,July 2025,barang original enak di pakek ngebut sepatunya,5,0
2,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,R***i,None,NaN,July 2025,Bagus nyaman melebihi ekspektasi saya,5,0
3,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,V***u,None,NaN,July 2025,Produknya mantapppplll,5,1
4,910,nineten haze strike prime,sepatu lari,Rp550.905,550905,Y***e,None,NaN,July 2025,Pengiriman cepat,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
13450,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,G***A,None,41.0,November 2024,Sepatunya bagus bgt...,5,0
13451,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,ì***¸,None,42.0,April 2024,Good,4,0
13452,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,2***«,None,41.0,April 2024,sumpah ini keren bngtttt,5,0
13453,Nuevo,nuevo siloka shadow low,sepatu lifestyle,Rp59.900,59900,d***a,None,42.0,April 2024,Lumyan,5,0
